In [1]:
import findspark
findspark.init()
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.ml.classification import LogisticRegression


In [2]:
spark = SparkSession \
    .builder \
    .appName("TFM_EDA") \
    .getOrCreate()

In [3]:
schema = StructType([StructField("Time",DoubleType(),True),StructField("V1",DoubleType(),True),StructField("V2",DoubleType(),True),StructField("V3",DoubleType(),True),
StructField("V4",DoubleType(),True),StructField("V5",DoubleType(),True),StructField("V6",DoubleType(),True),StructField("V7",DoubleType(),True),StructField("V8",DoubleType(),True),
StructField("V9",DoubleType(),True),StructField("V10",DoubleType(),True),StructField("V11",DoubleType(),True),StructField("V12",DoubleType(),True),StructField("V13",DoubleType(),True),
StructField("V14",DoubleType(),True),StructField("V15",DoubleType(),True),StructField("V16",DoubleType(),True),StructField("V17",DoubleType(),True),StructField("V18",DoubleType(),True),
StructField("V19",DoubleType(),True),StructField("V20",DoubleType(),True),StructField("V21",DoubleType(),True),StructField("V22",DoubleType(),True),StructField("V23",DoubleType(),True),
StructField("V24",DoubleType(),True),StructField("V25",DoubleType(),True),StructField("V26",DoubleType(),True),StructField("V27",DoubleType(),True),StructField("V28",DoubleType(),True),
StructField("Amount",DoubleType(),True),StructField("Class",IntegerType(),True)])

In [5]:
df_raw = spark.read.format("csv")\
        .option("header","True")\
        .option("inferSchema","True")\
        .schema(schema)\
        .load("creditcard.csv")

In [6]:
df_raw.take(1)

[Row(Time=0.0, V1=-1.3598071336738, V2=-0.0727811733098497, V3=2.53634673796914, V4=1.37815522427443, V5=-0.338320769942518, V6=0.462387777762292, V7=0.239598554061257, V8=0.0986979012610507, V9=0.363786969611213, V10=0.0907941719789316, V11=-0.551599533260813, V12=-0.617800855762348, V13=-0.991389847235408, V14=-0.311169353699879, V15=1.46817697209427, V16=-0.470400525259478, V17=0.207971241929242, V18=0.0257905801985591, V19=0.403992960255733, V20=0.251412098239705, V21=-0.018306777944153, V22=0.277837575558899, V23=-0.110473910188767, V24=0.0669280749146731, V25=0.128539358273528, V26=-0.189114843888824, V27=0.133558376740387, V28=-0.0210530534538215, Amount=149.62, Class=0)]

In [7]:
df_raw.describe().toPandas()

,summary,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,count,284807,284807,284807,284807,284807,284807,284807,284807,284807,...,284807,284807,284807,284807,284807,284807,284807,284807,284807,284807
1,mean,94813.85957508067,9.516248586879277E-16,-4.151383611725859E-17,-1.3156692677161953E-15,1.4976914722303291E-15,8.941441625255696E-16,1.3971002539462024E-15,-2.6664656275316093E-16,1.528826849318273E-16,...,3.718282533002536E-16,-4.630389413078843E-16,2.9858028284335985E-16,4.504351011597839E-15,7.153153300204557E-16,1.6364035688721307E-15,-3.568593220079729E-16,-1.2593860860572196E-16,88.34961925093698,0.001727485630620034
2,stddev,47488.145954566266,1.9586958038574889,1.6513085794769968,1.5162550051777701,1.4158685749409217,1.3802467340314388,1.332271089757576,1.2370935981826656,1.194352902669203,...,0.734524014371313,0.725701560440911,0.6244602955949902,0.6056470678271607,0.5212780705409428,0.48222701326105727,0.4036324949650301,0.3300832641602508,250.12010924018836,0.041527189635464895
3,min,0.0,-56.407509631329,-72.7157275629303,-48.3255893623954,-5.68317119816995,-113.743306711146,-26.1605059358433,-43.5572415712451,-73.2167184552674,...,-34.8303821448146,-10.933143697655,-44.8077352037913,-2.83662691870341,-10.2953970749851,-2.60455055280817,-22.5656793207827,-15.4300839055349,0.0,0
4,max,172792.0,2.45492999121121,22.0577289904909,9.38255843282114,16.8753440335975,34.8016658766686,73.3016255459646,120.589493945238,20.0072083651213,...,27.2028391573154,10.5030900899454,22.5284116897749,4.58454913689817,7.51958867870916,3.5173456116238,31.6121981061363,33.8478078188831,25691.16,1


In [8]:
df_raw.groupBy("Class").count().withColumn("Porcentaje", round(((col("count")/df_raw.count())*100),3)).show()

+-----+------+----------+
|Class| count|Porcentaje|
+-----+------+----------+
|    1|   492|     0.173|
|    0|284315|    99.827|
+-----+------+----------+



In [47]:
def correlation(df, pr, sc):
    temp=[]
    for a in sc:
        temp.append([a,df.corr(pr,a)])
    return pd.DataFrame(temp,columns=["variable","corr"])
udf_correlation = udf(correlation)

In [48]:
columns = df_raw.columns[:-1]

arra_corr= correlation(df_raw,"Class",columns)

In [51]:
arra_corr

,variable,corr
0,Time,-0.012323
1,V1,-0.101347
2,V2,0.091289
3,V3,-0.192961
4,V4,0.133447
5,V5,-0.094974
6,V6,-0.043643
7,V7,-0.187257
8,V8,0.019875
9,V9,-0.097733


In [12]:
df_label_1 = df_raw.where(col("Class") == 1)
df_label_0 = df_raw.where(col("Class") == 0).sample(False, df_label_1.count()/df_raw.count(),42)
df_final = df_label_1.union(df_label_0).orderBy(rand())

In [23]:
# Agrupamos las columnas que usaremos para entrenar nuestro modelo
feature_columns = [col for col in df_final.columns if col.startswith("V")]
vectorizer = VectorAssembler(inputCols = feature_columns, outputCol="features")

In [24]:
df_train, df_test = df_final.randomSplit([0.8, 0.2], seed = 123)

In [25]:
from pyspark.ml.classification import RandomForestClassifier

In [26]:
# Declaramos nuestro algoritmo 
est = RandomForestClassifier()
est.setFeatureSubsetStrategy("auto")
est.setLabelCol("Class")
est.setSeed(42)

RandomForestClassifier_dc5b092c79b1

In [27]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler 
from pyspark.sql.functions import *
from sklearn.metrics import classification_report, confusion_matrix
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel

# Conectamos mediante una tuberia nuestro vectorizer y nuestro algoritmo
pipeline = Pipeline()
pipeline.setStages([vectorizer, est])
# Entrenamos el algoritmo
model = pipeline.fit(df_train)

In [28]:
df_test_pred = model.transform(df_test)

In [29]:
# Creamos la Matrix de cunfusion 
y_true = df_test_pred.select(['Class']).collect()
y_pred = df_test_pred.select(['prediction']).collect()
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94       102
           1       0.95      0.91      0.93        90

    accuracy                           0.94       192
   macro avg       0.94      0.94      0.94       192
weighted avg       0.94      0.94      0.94       192



In [33]:
# Comenzamos a tunear el CrossValidator

paramGrid = ParamGridBuilder()\
            .addGrid(est.maxBins,[10,50,100])\
            .addGrid(est.impurity,["entropy","gini"])\
            .addGrid(est.maxDepth,[4,6,8])\
            .build()

evaluator = BinaryClassificationEvaluator()\
            .setLabelCol("Class")\
            .setMetricName("areaUnderPR")


crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2)

In [34]:
# Entrenamos y realizamos las predicciones
cvModel = crossval.fit(df_train)
df_test_pred = cvModel.transform(df_test)

In [35]:
# Introducimos nuestras predicciones en la matrix de confusion y observamos una mejora de nuestras metricas. 
y_true = df_test_pred.select(['Class']).collect()
y_pred = df_test_pred.select(['prediction']).collect()
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94       102
           1       0.93      0.93      0.93        90

    accuracy                           0.94       192
   macro avg       0.94      0.94      0.94       192
weighted avg       0.94      0.94      0.94       192



In [ ]:
# Guardamos el Modelo para exportarlo a nuestro archivo de Spark Streaming. 
model = cvModel.bestModel
model.write().overwrite().save("credit-model")